In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

# Dataset path
path_to_datasets = "/content/gdrive/My Drive/Colab Notebooks/CS4248Project"

# Importing the required modules
import numpy as np
import pandas as pd
import re
import string
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

Mounted at /content/gdrive


In [2]:
train = pd.read_csv(f"{path_to_datasets}/fulltrain.csv", names = ['label', 'text'])
train.head()

,label,text
0,1,"A little less than a decade ago, hockey fans w..."
1,1,The writers of the HBO series The Sopranos too...
2,1,Despite claims from the TV news outlet to offe...
3,1,After receiving 'subpar' service and experienc...
4,1,After watching his beloved Seattle Mariners pr...


In [ ]:
# # Using the preprocessing utils implemented by the other team members

# # remove punctuation
# train['text'] = train['text'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '' , x))

# # lower case
# def lower_case(sentence):
#     sentence = str(sentence)
#     sentence = sentence.lower()
#     return sentence
# train['text'] = train['text'].apply(lambda x:lower_case(x))

# train.head()

In [ ]:
# # Test train split

# from sklearn.model_selection import train_test_split

# X = train['text']
# y = train['label']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.5)

In [ ]:
# # Tokenising

# import tensorflow as tf

# tokenizer = tf.keras.layers.TextVectorization(
#     standardize='lower_and_strip_punctuation',
#     output_mode='int'
# )

# tokenizer.adapt(X_train)
# size_train = tokenizer.vocabulary_size()
# vocabulary_train = tokenizer.get_vocabulary()

# tokenizer.adapt(X_test)
# size_test = tokenizer.vocabulary_size()
# vocabulary_test = tokenizer.get_vocabulary()

# print(size_train)
# print(size_test)
# print(len(X_train))

In [ ]:
# # Using Glove

# embeddings_dict = {}

# with open("/content/gdrive/My Drive/Colab Notebooks/CS4248Project/glove.6B.100d.txt", 'r') as f:
#   for line in f:
#         values = line.split()
#         word = values[0]
#         vector = np.asarray(values[1:], "float32")
#         embeddings_dict[word] = vector

# #TODO: replace size_train by X_train_size
# embedding_matrix_train = np.zeros((len(X_train), 100))
# embedding_matrix_test = np.zeros((len(X_test), 100))

# for k, w in enumerate(vocabulary_train):
#     embedding_mat = embeddings_dict.get(w)
#     if embedding_mat is not None:
#         embedding_matrix_train[k] = embedding_mat

# for k, w in enumerate(vocabulary_test):
#     embedding_mat = embeddings_dict.get(w)
#     if embedding_mat is not None:
#         embedding_matrix_test[k] = embedding_mat

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

X = train['text']
y = train['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

# Using tfidf and one-hot encoding to fit and transform the data
Tfidf_vect = TfidfVectorizer(sublinear_tf=True, max_df=0.5, min_df=5, stop_words="english")
Tfidf_vect.fit(X_train)
# Train
Train_X_Tfidf = Tfidf_vect.transform(X_train)
# Test
Test_X_Tfidf =Tfidf_vect.transform(X_test)

In [ ]:
# # Multi layer perceptron

# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Activation
# import keras
# import tensorflow as tf

# output_layer_cat = 4
# features = embedding_matrix_train.shape[1]
# length = 256
# runs = 10

# Y_train = tf.keras.utils.to_categorical(y_train-1)
# Y_test = tf.keras.utils.to_categorical(y_test-1)

# model = Sequential()
# model.add(Dense(512, input_shape=(features,)))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))
# model.add(Dense(output_layer_cat))
# model.add(Activation('sigmoid'))
# model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
# history = model.fit(embedding_matrix_train, Y_train, batch_size = length, epochs = runs, verbose = 1, validation_split = 0.1)

# score = model.evaluate(embedding_matrix_test, Y_test, batch_size = length, verbose = 1)

# print('Accuracy:', score[1])

In [ ]:
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Activation
# import keras

# output_layer_cat = 4
# features = Train_X_Tfidf.shape[1]
# length = 256
# runs = 3

# Y_train = keras.utils.to_categorical(y_train2)
# Y_test = keras.utils.to_categorical(y_test2)

# model = Sequential()
# model.add(Dense(512, input_shape=(features,)))
# model.add(Activation('tanh'))
# model.add(Dropout(0.5))
# model.add(Dense(output_layer_cat))
# model.add(Activation('softmax'))
# model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
# history = model.fit(Train_X_Tfidf.toarray(), Y_train, batch_size = length, epochs = runs, verbose = 1, validation_split = 0.1)

# score = model.evaluate(Test_X_Tfidf.toarray(), Y_test, batch_size = length, verbose = 1)

# print('Accuracy:', score[1])

In [ ]:
# Logistic regression with new td-idf

from sklearn.linear_model import LogisticRegression
from sklearn import metrics

lr = LogisticRegression(C=100.0, random_state=1, solver='lbfgs', multi_class='ovr', max_iter=50000)

lr.fit(Train_X_Tfidf, y_train)

prediction_train = lr.predict(Train_X_Tfidf)
prediction_test = lr.predict(Test_X_Tfidf)

print("LogisticRegression with tf-idf Accuracy %.3f for Training Data " %metrics.accuracy_score(prediction_train, y_train))
print("LogisticRegression with tf-idf Accuracy %.3f for Test Data " %metrics.accuracy_score(prediction_test, y_test))

LogisticRegression with tf-idf Accuracy 1.000 for Training Data 
LogisticRegression with tf-idf Accuracy 0.972 for Test Data 


In [ ]:
# Logistic regression with new td-idf

from sklearn.linear_model import LogisticRegression
from sklearn import metrics

lr = LogisticRegression(C=100.0, random_state=1, solver='lbfgs', multi_class='ovr', max_iter=50000)

lr.fit(Train_X_Tfidf, y_train)

prediction_train = lr.predict(Train_X_Tfidf)
prediction_test = lr.predict(Test_X_Tfidf)

print("LogisticRegression with tf-idf Accuracy %.3f for Training Data " %metrics.accuracy_score(prediction_train, y_train))
print("LogisticRegression with tf-idf Accuracy %.3f for Test Data " %metrics.accuracy_score(prediction_test, y_test))
print("LogisticRegression with tf-idf f1 score %.3f for Test Data " %metrics.f1_score(prediction_test, y_test, average='macro'))

LogisticRegression with tf-idf Accuracy 1.000 for Training Data 
LogisticRegression with tf-idf Accuracy 0.972 for Test Data 
LogisticRegression with tf-idf f1 score 0.972 for Test Data 


In [ ]:
# Naive bayes with tf-idf

from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

nb = MultinomialNB(force_alpha=True)

nb.fit(Train_X_Tfidf, y_train)

prediction_train = nb.predict(Train_X_Tfidf)
prediction_test = nb.predict(Test_X_Tfidf)

print("Naive Bayes with tf-idf Accuracy %.3f for Training Data " %metrics.accuracy_score(prediction_train, y_train))
print("Naive Bayes with tf-idf Accuracy %.3f for Test Data " %metrics.accuracy_score(prediction_test, y_test))

Naive Bayes with tf-idf Accuracy 0.878 for Training Data 
Naive Bayes with tf-idf Accuracy 0.855 for Test Data 


In [ ]:
# Naive bayes with tf-idf

from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

nb = MultinomialNB(force_alpha=True)

nb.fit(Train_X_Tfidf, y_train)

prediction_train = nb.predict(Train_X_Tfidf)
prediction_test = nb.predict(Test_X_Tfidf)

print("Naive Bayes with tf-idf Accuracy %.3f for Training Data " %metrics.accuracy_score(prediction_train, y_train))
print("Naive Bayes with tf-idf Accuracy %.3f for Test Data " %metrics.accuracy_score(prediction_test, y_test))
print("Naive Bayes with tf-idf f1 score %.3f for Test Data " %metrics.f1_score(prediction_test, y_test, average='macro'))

Naive Bayes with tf-idf Accuracy 0.878 for Training Data 
Naive Bayes with tf-idf Accuracy 0.854 for Test Data 
Naive Bayes with tf-idf Accuracy 0.839 for Test Data 
